In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from data_util import OptionDataset
from hyperiv_util import SetEmbeddingNetwork, HyperNetwork
from trainer_util import trainer
import torch.optim as optim

In [2]:
df = pd.read_hdf('spx_w_ref.h5', 'df')

In [3]:
N = 1024
B = 128

train_split = "2023-01-01"

train_dates = df[df["date"] < train_split]["date"].unique()
test_dates = df[df["date"] >= train_split]["date"].unique()

train_dataset = OptionDataset(df[df["date"].isin(train_dates)], N=N, sample=True)
test_dataset = OptionDataset(df[df["date"].isin(test_dates)], N=N, sample=False)

train_dataloader = DataLoader(train_dataset, batch_size=B, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, drop_last=False)

In [4]:
iv_network = torch.nn.Sequential(
    torch.nn.Linear(2, 16),
    torch.nn.Tanh(),
    torch.nn.Linear(16, 16),
    torch.nn.Tanh(),
    torch.nn.Linear(16, 1),
    torch.nn.Softplus()
)

In [5]:
n_params = sum([p.numel() for p in iv_network.parameters()])
n_params

337

In [6]:
input_dim = 3
output_dim = n_params

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hyper_model = SetEmbeddingNetwork(input_dim, output_dim).to(device)

model = HyperNetwork(hyper_model, iv_network)

optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 500
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs, eta_min=1e-5)

In [7]:
for epoch in range(num_epochs):
    train_loss_mse, train_loss_mae, train_loss_cal, train_loss_g, train_loss_integral = trainer(train_dataloader, model, device, optimizer, is_train=True)
    print(f'Epoch {epoch+1}/{num_epochs}, Train MSE: {train_loss_mse:.8f}, Train MAE: {train_loss_mae:.8f}, Train CAL: {train_loss_cal:.8f}, Train G: {train_loss_g:.8f}, Train Integral: {train_loss_integral:.8f}')
    lr_scheduler.step()
    test_loss_mse, test_loss_mae, test_loss_cal, test_loss_g, test_loss_integral = trainer(test_dataloader, model, device, optimizer, is_train=False)
    print(f'Epoch {epoch+1}/{num_epochs}, Test MSE: {test_loss_mse:.8f}, Test MAE: {test_loss_mae:.8f}, Test CAL: {test_loss_cal:.8f}, Test G: {test_loss_g:.8f}, Test Integral: {test_loss_integral:.8f}')

100%|██████████| 19/19 [01:28<00:00,  4.64s/it]


Epoch 1/500, Train MSE: 0.57586560, Train MAE: 0.30975822, Train CAL: 0.00310543, Train G: 0.01879865, Train Integral: 0.06096550


100%|██████████| 167/167 [00:01<00:00, 90.04it/s]


Epoch 1/500, Test MSE: 0.01333086, Test MAE: 0.07582381, Test CAL: 0.00000000, Test G: 0.00000000, Test Integral: 0.00093144


100%|██████████| 19/19 [01:29<00:00,  4.71s/it]


...

Epoch 3/500, Train MSE: 0.00389524, Train MAE: 0.04063119, Train CAL: 0.00003844, Train G: 0.00003252, Train Integral: 0.00008203


100%|██████████| 167/167 [00:01<00:00, 93.23it/s]


Epoch 3/500, Test MSE: 0.00193146, Test MAE: 0.03360750, Test CAL: 0.00000000, Test G: 0.00000000, Test Integral: 0.00005192


 26%|██▋       | 5/19 [00:24<01:07,  4.84s/it]


KeyboardInterrupt: 

In [8]:
torch.save(hyper_model.state_dict(), 'spx_hyperiv.pth')